# Documentation

https://opennmt.net/CTranslate2/quickstart.html  
https://opennmt.net/CTranslate2/guides/opus_mt.html  

# Model Conversion

In [ ]:
!pip install torch --index-url https://download.pytorch.org/whl/cpu
!pip install ctranslate2 OpenNMT-py==2.* sentencepiece

# 1m

In [2]:
!wget https://object.pouta.csc.fi/OPUS-MT-models/en-de/opus-2020-02-26.zip
!unzip opus-2020-02-26.zip -d /tmp/opus_mt_en_de "*.npz" "*.spm" "*.yml"

--2024-04-05 17:04:23--  https://object.pouta.csc.fi/OPUS-MT-models/en-de/opus-2020-02-26.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275458389 (263M) [application/zip]
Saving to: ‘opus-2020-02-26.zip’

opus-2020-02-26.zip 100%[===================>] 262.70M  19.9MB/s    in 14s     

2024-04-05 17:04:38 (18.9 MB/s) - ‘opus-2020-02-26.zip’ saved [275458389/275458389]

Archive:  opus-2020-02-26.zip
  inflating: /tmp/opus_mt_en_de/opus.spm32k-spm32k.transformer-align.model1.npz.best-perplexity.npz  
  inflating: /tmp/opus_mt_en_de/opus.spm32k-spm32k.vocab.yml  
  inflating: /tmp/opus_mt_en_de/source.spm  
  inflating: /tmp/opus_mt_en_de/target.spm  
  inflating: /tmp/opus_mt_en_de/decoder.yml  


In [3]:
!ls -lh /tmp/opus_mt_en_de

total 285M
-rw-rw---- 1 root root  265 Feb 26  2020 decoder.yml
-rw-rw-r-- 1 root root 283M Feb 22  2020 opus.spm32k-spm32k.transformer-align.model1.npz.best-perplexity.npz
-rw-rw-r-- 1 root root 913K Feb 17  2020 opus.spm32k-spm32k.vocab.yml
-rw-rw---- 1 root root 751K Feb 26  2020 source.spm
-rw-rw---- 1 root root 779K Feb 26  2020 target.spm


In [4]:
!ct2-opus-mt-converter --model_dir /tmp/opus_mt_en_de --output_dir /tmp/ctranslate_2_en_de

In [5]:
!cp /tmp/opus_mt_en_de/source.spm /tmp/ctranslate_2_en_de/source.spm

In [6]:
!ls -lh /tmp/ctranslate_2_en_de

total 288M
-rw-r--r-- 1 root root  221 Apr  5 17:05 config.json
-rw-r--r-- 1 root root 287M Apr  5 17:05 model.bin
-rw-r--r-- 1 root root 971K Apr  5 17:05 shared_vocabulary.json
-rw-r----- 1 root root 751K Apr  5 17:05 source.spm


# Docker Container Preparation Script

In [7]:
from nltk import download as nltk_download

nltk_download(
    'punkt',
    download_dir='/usr/share/nltk_data'
)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Translation Script

In [8]:
from multiprocessing import cpu_count

import ctranslate2
from nltk.tokenize import sent_tokenize
import sentencepiece as spm

In [28]:
translator_model = ctranslate2.Translator(
    '/tmp/ctranslate_2_en_de',
    device='cpu',
    inter_threads=1,
    intra_threads=cpu_count()
)

sentence_piece = spm.SentencePieceProcessor('/tmp/ctranslate_2_en_de/source.spm')

In [20]:
def translator(input_text: str) -> str:
    input_tokens = sentence_piece.encode(input_text, out_type=str)
    results = translator_model.translate_batch([input_tokens])

    output_tokens = results[0].hypotheses[0]
    output_text = sentence_piece.decode(output_tokens)

    # Remove conrol character: U+2581 : LOWER ONE EIGHTH BLOCK:
    output_text = output_text.replace('▁', ' ')

    return output_text

In [12]:
input_text = (
    'London is the capital and largest city of England, and the United Kingdom, with a population of around 8.8 million, ' +
    'and the largest city in Western Europe by metropolitan area, with a population of 14.8 million. ' +
    'It stands on the River Thames in south-east England at the head of a 50-mile (80 km) estuary down to the North Sea and ' +
    'has been a major settlement for nearly two millennia. The City of London, its ancient core and financial centre, ' +
    'was founded by the Romans as Londinium and retains its medieval boundaries. The City of Westminster, ' +
    'to the west of the City of London, has for centuries hosted the national government and parliament. ' +
    'Since the 19th century, the name "London" also refers to the metropolis around this core, ' +
    'historically split among the counties of Middlesex, Essex, Surrey, Kent, and Hertfordshire, ' +
    'which since 1965 has largely comprised Greater London, which is governed by 33 local authorities and ' +
    'the Greater London Authority.'
)

In [29]:
translator(input_text)

# 13s

'London ist die Hauptstadt und größte Stadt Englands, und das Vereinigte Königreich mit einer Bevölkerung von rund 8,8 Millionen und die größte Stadt Westeuropas durch Metropolregion, mit einer Bevölkerung von 14,8 Millionen. Es steht an der Themse im Südosten Englands an der Spitze einer 50-Meilen (80 km) Mündung bis zur Nordsee und ist seit fast zwei Jahrtausenden eine große Siedlung. Die City of London, ihr alter Kern und Finanzzentrum, wurde von den Römern als Londinium gegründet und behält seine mittelalterlichen Grenzen. Die City of Westminster, im Westen der City of London, beherbergt seit Jahrhunderten die nationale Regierung und das Parlament. Seit dem 19. Jahrhundert, der Name "London" bezieht sich auch auf die Metropole um diesen Kern, historisch aufgeteilt zwischen den Grafschaften Middlesex, Essex, Surrey, Kent und Hertfordshire, die seit 1965 weitgehend Greater London, die von 33 lokalen Behörden und der Greater London Authority regiert.'

In [22]:
sentences = sent_tokenize(input_text)

In [30]:
for sentence in sentences:
    print(sentence)
    print(translator(sentence))
    print('')

# 13s

London is the capital and largest city of England, and the United Kingdom, with a population of around 8.8 million, and the largest city in Western Europe by metropolitan area, with a population of 14.8 million.
London ist die Hauptstadt und größte Stadt Englands, und das Vereinigte Königreich mit einer Bevölkerung von etwa 8,8 Millionen und die größte Stadt Westeuropas nach Metropolregionen, mit einer Bevölkerung von 14,8 Millionen.

It stands on the River Thames in south-east England at the head of a 50-mile (80 km) estuary down to the North Sea and has been a major settlement for nearly two millennia.
Sie liegt an der Themse im Südosten Englands an der Spitze einer 80 km langen Mündung bis zur Nordsee und ist seit fast zwei Jahrtausenden eine bedeutende Siedlung.

The City of London, its ancient core and financial centre, was founded by the Romans as Londinium and retains its medieval boundaries.
Die City of London, ihr altes Kern- und Finanzzentrum, wurde von den Römern als Londini